In [1]:
import util
import stac
from dm_control import mujoco
from dm_control import mjcf
from dm_control import suite
from dm_control import viewer
from dm_control import composer
from dm_control.rl import control
from dm_control.locomotion.walkers import base
import multiprocessing as mp

from dm_control.mujoco.wrapper.mjbindings import mjlib
import numpy as np
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import imp 
import rodent_environments
imp.reload(util)
imp.reload(rodent_environments)
imp.reload(stac)

<module 'stac' from '/home/diego/code/olveczky/stac/stac.py'>

# Load Data and model 

In [ ]:
# Pathing
model_path = "/home/diego/code/olveczky/stac/ratMocap/models/rat.xml"
data_path = "/home/diego/code/olveczky/stac/ratMocap/dataInput/nolj_Recording_day2_overnight5_nolj_imputed_JDM33.mat"

# Load
kp_data, kp_names = util.load_kp_data_from_file(data_path, struct_name='markers_aligned_preproc');
print("Keypoint data is shape: ", kp_data.shape)
print("Keypoint names are: ", kp_names)
kp_data = kp_data[100000::10,:]/1000
# Construct model and physics
rat = util.RatWithMocapSites(model_path, kp_data, kp_names)
skeleton_spine_length = np.sqrt(np.sum((rat.physics.named.data.xpos[2] - rat.physics.named.data.xpos[4])**2))
mocap_spine_length = np.sqrt(np.sum((np.mean(kp_data, axis=0)[9:12] - np.mean(kp_data, axis=0)[12:15])**2))
factor = mocap_spine_length/skeleton_spine_length
factor = 1.9
kp_data *= factor
kp_data[:,2::3] += rat.physics.named.data.xpos[4][2] + .01
print(factor)



In [2]:
# Load
# data_path = "/home/diego/code/olveczky/stac/ratMocap/dataInput/nolj_Recording_day2_overnight5_nolj_imputed_JDM33.mat"
data_path = "/home/diego/code/olveczky/stac/ratMocap/dataInput/nolj_Recording_day8_caff3_nolj_imputed.mat"
kp_data, kp_names = util.load_kp_data_from_file(data_path, struct_name='markers_preproc');
print("Keypoint data is shape: ", kp_data.shape)
print("Keypoint names are: ", kp_names)
factor = 1.5
start_frame = 30900
skip = 10
kp_data = kp_data[start_frame::skip,:]/1000*factor
kp_data[:,2::3] -= .02

Keypoint data is shape:  (540000, 60)
Keypoint names are:  ['ArmL', 'ArmR', 'ElbowL', 'ElbowR', 'HeadB', 'HeadF', 'HeadL', 'HipL', 'HipR', 'KneeL', 'KneeR', 'Offset1', 'Offset2', 'ShinL', 'ShinR', 'ShoulderL', 'ShoulderR', 'SpineF', 'SpineL', 'SpineM']


# Get initial Dm

In [ ]:
imp.reload(stac)
imp.reload(rodent_environments)
env = rodent_environments.rodent_mocap(kp_data)
viewer.launch(env)

In [ ]:
imp.reload(rodent_environments)
imp.reload(stac)

# Params
_IS_LIMB = ['scapula', 'hip', 'toe', 'knee', 'ankle', 'shoulder', 'finger', 'elbow', 'wrist']
n_sample_frames = 50
n_frames_clip = 500
n_iters = 1
q_reg_coef = 0.
m_reg_coef = .1

# Initialize the rodent and environment 
env = rodent_environments.rodent_mocap(kp_data, n_frames_clip)
initial_offsets = np.copy(env.physics.bind(env.task._walker.body_sites).pos[:])
limbs = np.zeros_like(env.physics.named.data.qpos[:], dtype='int32')
for i, name in enumerate(env.physics.named.data.qpos.axes.row.names):
    if any(part in name for part in _IS_LIMB):
        limbs[i] = 1
head_body = ~limbs
# Iterate between q and m phases 
q = []
for iteration in range(n_iters):
    q = []
    # Optimize the root of the first frame
    stac.q_phase(env.physics, env.task.kp_data[0,:], env.task._walker.body_sites, root_only=True)
#     stac.q_phase(env.physics, env.task.kp_data[0,:], 0, env.task._walker.body_sites)
    q.append(env.physics.named.data.qpos[:].copy())
    
    # m on first 
    stac.m_phase(env.physics, env.task.kp_data, env.task._walker.body_sites, [0], q, initial_offsets, reg_coef=m_reg_coef, maxiter=20)
    q = []
    
    # Find the frames to use in the optimization.
    time_indices = np.random.randint(0, n_frames_clip, n_sample_frames)
    
    # Q-phase across the clip: optimize joint angles
    print('q-phase')
    for i in range(n_frames_clip):
        if np.mod(i,10) == 0: print(i)
        stac.q_phase(env.physics, env.task.kp_data[i,:], env.task._walker.body_sites, reg_coef=q_reg_coef)
        stac.q_phase(env.physics, env.task.kp_data[i,:], env.task._walker.body_sites, reg_coef=q_reg_coef, qs_to_opt=limbs)
        retval = env.physics.named.data.qpos[:]
        q.append(retval.copy())
    
    # M-phase across the subsampling: optimize offsets
    print('m-phase')
    stac.m_phase(env.physics, env.task.kp_data, env.task._walker.body_sites, time_indices, q, initial_offsets, reg_coef=m_reg_coef)
    
    # Q-phase across the clip: optimize joint angles
    q = []
    x = []
    print('q-phase')
    for i in range(n_frames_clip):
        if np.mod(i,10) == 0: print(i)
        stac.q_phase(env.physics, env.task.kp_data[i,:], env.task._walker.body_sites, reg_coef=q_reg_coef)
        stac.q_phase(env.physics, env.task.kp_data[i,:], env.task._walker.body_sites, reg_coef=q_reg_coef, qs_to_opt=limbs)
        retval = env.physics.named.data.qpos[:]
        q.append(retval.copy())
        retval = env.physics.named.data.xpos[:]
        x.append(retval.copy())

# Prep the environment for updating the body
env.task.precomp_qpos = q
env.task.precomp_xpos = x

# GO! 
env.task.render_video=True
viewer.launch(env)
env.task.V.release()

q-phase
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320


In [145]:

env.physics.named.model._asdict()['cam_bodyid']

FieldIndexer(cam_bodyid):
0        top_camera [0]
1       walker/back [2]
2       walker/side [2]
3 walker/egocentric [55]

In [229]:
imp.reload(rodent_environments)
env = rodent_environments.rodent_mocap(kp_data, n_frames_clip)
env.task.render_video=False
env.task.precomp_qpos = q
env.task.precomp_xpos = x
env.task.V = None
viewer.launch(env)
env.task.V.release()

AttributeError: 'NoneType' object has no attribute 'release'

In [ ]:
env.task

In [207]:
env.task.V.release()

In [225]:
env.physics.named.data.xpos.axes.row.names

['world',
 'rat/',
 'rat/torso',
 'rat/vertebra_1',
 'rat/vertebra_2',
 'rat/vertebra_3',
 'rat/vertebra_4',
 'rat/vertebra_5',
 'rat/vertebra_6',
 'rat/pelvis',
 'rat/upper_leg_L',
 'rat/lower_leg_L',
 'rat/foot_L',
 'rat/toe_L',
 'rat/upper_leg_R',
 'rat/lower_leg_R',
 'rat/foot_R',
 'rat/toe_R',
 'rat/vertebra_C1',
 'rat/vertebra_C2',
 'rat/vertebra_C3',
 'rat/vertebra_C4',
 'rat/vertebra_C5',
 'rat/vertebra_C6',
 'rat/vertebra_C7',
 'rat/vertebra_C8',
 'rat/vertebra_C9',
 'rat/vertebra_C10',
 'rat/vertebra_C11',
 'rat/vertebra_C12',
 'rat/vertebra_C13',
 'rat/vertebra_C14',
 'rat/vertebra_C15',
 'rat/vertebra_C16',
 'rat/vertebra_C17',
 'rat/vertebra_C18',
 'rat/vertebra_C19',
 'rat/vertebra_C20',
 'rat/vertebra_C21',
 'rat/vertebra_C22',
 'rat/vertebra_C23',
 'rat/vertebra_C24',
 'rat/vertebra_C25',
 'rat/vertebra_C26',
 'rat/vertebra_C27',
 'rat/vertebra_C28',
 'rat/vertebra_C29',
 'rat/vertebra_C30',
 'rat/vertebra_cervical_5',
 'rat/vertebra_cervical_4',
 'rat/vertebra_cervical

In [108]:
env.physics.named.data.xpos[:].shape

(67, 3)

In [84]:
# env.task._walker._mjcf_root.find_all('body')
if 'yellow' in ['yellow', 'red']: print('yes')
env.physics.named.data.qpos.axes.row.names

yes


['walker/',
 'walker/vertebra_1_extend',
 'walker/vertebra_2_bend',
 'walker/vertebra_3_twist',
 'walker/vertebra_4_extend',
 'walker/vertebra_5_bend',
 'walker/vertebra_6_twist',
 'walker/hip_L_supinate',
 'walker/hip_L_abduct',
 'walker/hip_L_extend',
 'walker/knee_L',
 'walker/ankle_L',
 'walker/toe_L',
 'walker/hip_R_supinate',
 'walker/hip_R_abduct',
 'walker/hip_R_extend',
 'walker/knee_R',
 'walker/ankle_R',
 'walker/toe_R',
 'walker/vertebra_C1_extend',
 'walker/vertebra_C3_bend',
 'walker/vertebra_C5_extend',
 'walker/vertebra_C7_bend',
 'walker/vertebra_C9_extend',
 'walker/vertebra_C11_bend',
 'walker/vertebra_C13_extend',
 'walker/vertebra_C15_bend',
 'walker/vertebra_C17_extend',
 'walker/vertebra_C19_bend',
 'walker/vertebra_C21_extend',
 'walker/vertebra_C23_bend',
 'walker/vertebra_C25_extend',
 'walker/vertebra_C27_bend',
 'walker/vertebra_C29_extend',
 'walker/vertebra_cervical_5_extend',
 'walker/vertebra_cervical_4_bend',
 'walker/vertebra_cervical_3_twist',
 'walke

In [16]:
env = rodent_environments.rodent_mocap(kp_data)
viewer.launch(env)

In [ ]:
np.stack([np.array([1,2,3]),np.array([1,2,3])])

In [112]:
viewer.launch(env)

E0513 21:05:38.156582 140346327054144 util.py:312] Domain Explorer intercepted an environment error.
Original message: list index out of range
Domain Explorer intercepted an environment error.
Original message: list index out of range
Traceback:
  File "/home/diego/.envs/mujoco200_3.7/lib/python3.6/site-packages/dm_control/viewer/runtime.py", line 246, in _step
    self._time_step = self._env.step(action)
  File "/home/diego/.envs/mujoco200_3.7/lib/python3.6/site-packages/dm_control/composer/environment.py", line 378, in step
    self._hooks.after_step(self._physics_proxy, self._random_state)
  File "/home/diego/.envs/mujoco200_3.7/lib/python3.6/site-packages/dm_control/composer/environment.py", line 158, in after_step
    self._task.after_step(physics, random_state)
  File "/home/diego/code/olveczky/stac/rodent_environments.py", line 198, in after_step
    physics.named.data.qpos[:] = self.precomp_qpos[self.frame]
E0513 21:06:38.024388 140346327054144 util.py:312] Domain Explorer inte

In [ ]:
env.task._walker.body_sites

In [ ]:
print(env._task._arena.mjcf_model.worldbody.to_xml_string())

In [ ]:
imp.reload(stac)
rat = util.RatWithMocapSites(model_path, kp_data, kp_names)
env = control.Environment(rat.physics, suite.rat.Rat(0), time_limit=20, n_sub_steps=100, **{})
viewer.launch(env)

In [ ]:
imp.reload(stac)
rat = util.RatWithMocapSites(model_path, kp_data, kp_names)
stac.m_phase(rat.physics, kp_data, rat.get_sites(), n_frames=50)
env = control.Environment(rat.physics, suite.rat.Rat(0), time_limit=20, n_sub_steps=100, **{})
viewer.launch(env)

In [ ]:
env = control.Environment(rat.physics, suite.rat.Rat(0), time_limit=20, n_sub_steps=100, **{})
viewer.launch(env)

In [ ]:
env.physics.render()

# Run q_phase on a set of frames

In [ ]:
imp.reload(stac)
def stac_im(physics, kp_data, frame, sites):
    print(frame)
    stac.q_phase(physics, kp_data[frame,:], frame, sites)
    return physics.render(height=500,width=500)
def stac_qpos(physics, kp_data, frame, sites):
    print(frame)
    stac.q_phase(physics, kp_data[frame,:], frame, sites)
    retval = physics.named.data.qpos[:]
    return retval.copy()
V = [stac_im(env.physics, kp_data, frame, env.task._walker.body_sites) for frame in range(0,100,1)]

In [ ]:
def stac_qpos(physics, kp_data, frame, sites):
    if np.mod(frame, 10) == 0: print(frame)
    stac.q_phase(physics, kp_data[frame,:], frame, sites, reg_coef=0.01)
    retval = physics.named.data.qpos[:]
    return retval.copy()
Q = [stac_qpos(env.physics, kp_data, frame, env.task._walker.body_sites) for frame in range(0,1000,1)]

In [ ]:
env.task.precomp_qpos = Q
viewer.launch(env)

# View the results

In [ ]:
p = plt.figure()
vid = plt.imshow(V[0])
# initialization function: plot the background of each frame
def init():
    vid.set_data(V[0])
    return (vid,)

# animation function. This is called sequentially
def animate(i):
    vid.set_data(V[i])
    return (vid,)
anim = animation.FuncAnimation(p, animate, init_func=init,
                               frames=100, interval=1, blit=True)


In [ ]:
centered[0,:]

# Stac pipeline

In [ ]:
# env = suite.load(domain_name='rat', task_name='stand')
# viewer.launch(env)
imp.reload(util)
# rat = util.RatWithMocapSites(model_path, kp_data, kp_names)
env = control.Environment(rat.physics, suite.rat.Rat(0), time_limit=20, n_sub_steps=100, **{})
viewer.launch(env)

In [ ]:
env.action_spec()

In [ ]:
viewer.launch(env)

In [ ]:
env.__dict__

In [ ]:
t = mjcf.RootElement()
t.worldbody.add('site', type='sphere',size=[.1],rgba="1 0 0 1",pos=[0,0,0])
physics = mjcf.Physics.from_mjcf_model(t)
env = control.Environment(physics, suite.rat.Rat(0), time_limit=20, n_sub_steps=100, **{})
viewer.launch(env)

In [ ]:
imp.reload(stac)
def par_q_phase(physics, kp_data, frame, sites, queue):
    stac.q_phase(physics, kp_data, frame, sites)
    queue.put(physics.render(height=500,width=500,camera_id='side'))
output = mp.Queue()
processes = [mp.Process(target=par_q_phase, args=(rat.physics, kp_data[frame,:], frame, rat.get_sites(), output)) 
             for frame in range(0,100,10)]

for p in processes:
    p.start()
    
for p in processes:
    p.join()
    
results = [output.get() for p in processes]